# Research data supporting "Going for Gold(-Standard): Attaining Coupled Cluster Accuracy in Oxide-Supported Nanoclusters

This notebook accompanies our paper: **Going for Gold(-Standard): Attaining Coupled Cluster Accuracy in Oxide-Supported Nanoclusters**.

### Abstract

Metal nanoclusters supported on oxide surfaces are widely-used catalysts that boast sharply enhanced activity over their bulk, especially for the coinage metals: Au, Ag, and Cu. These properties depend sensitively on the nanocluster structure, which are challenging to model with density functional theory (DFT). Leveraging the recently developed SKZCAM protocol, we perform the first-ever benchmark study of coinage metal structures on the MgO surface with coupled cluster theory [CCSD(T)] -- the gold-standard modeling technique. Our benchmarks reveal that no DFT exchange-correlation functional (and dispersion correction) can accurately model this system. We demonstrate that this arises from inadequate account of metal-metal interactions and propose a high-level correction which provides reference accuracy at low cost. This forges a path towards studying larger systems, which we highlight by benchmarking Au20 on MgO, a challenging system where DFT disagrees significantly on its structure.


# Contents
* [Table S1 and Figure 1 - Past CO on MgO *E*<sub>ads</sub> and Current Work](#tables1fig1)
* [Table S2 and S3 - Validating the revPBE-D4 geometry and Computing its $\Delta_\textrm{geom}$](#tables2s3)
* [Table S4 and S5 - Periodic CCSD(T) Convergence](#tables4)
* [Table S6 - Periodic DMC Convergence](#tables6)
* [Table S7 - DFT Convergence](#tables7)
* [Table S9 to S11 and Figure 2 - SKZCAM Cluster CCSD(T) Convergence](#tables9-s11)
* [Table S8 - Final periodic CCSD(T), periodic DMC and cluster CCSD(T) $E_\textrm{ads}$ and their individual contributions](#tables8)
* [Table S12 - Analysis of previous computational work on CO on MgO](#tables12)
* [Table S13 - Computation of zero-point energy and thermal contribution terms to convert $H_\textrm{ads}$ to $E_\textrm{ads}$](#tables13)
* [Figure 3 and S1 - Converting previous experiment $H_\textrm{ads}$ or $E_\textrm{act}$ (for TPD) to $E_\textrm{ads}$](#fig3)
* [Table S14 - Final best estimate of TPD experimental adsorption energies](#tables14)
* [Table S15 - Effect of CO coverage on $E_\textrm{ads}$](#tables15)


In [3]:
# Check if we are in Google Colab environment
try:
    import google.colab
    IN_COLAB = True
    usetex = False
except:
    import os
    IN_COLAB = False
    if os.path.expanduser('~') == '/home/shixubenjamin':
        usetex = True
    else:
        usetex = False

# If in Google Colab, install the necessary data and set up the necessary environment
if IN_COLAB == True:
    !rm -rf /content/Data_CO_on_MgO-main /content/main.zip
    !wget https://github.com/benshi97/Data_CO_on_MgO/archive/refs/heads/main.zip
    !unzip /content/main.zip
    ! apt install ase
    !pip install pyblock datetime
    %pwd
    %cd /content/Data_CO_on_MgO-main

In [4]:
from ase import io
from ase.units import mol, kcal, kJ, Hartree
import pandas as pd
import pyblock
from Scripts.jup_plot import *
if usetex == True:
    textrue_import()
else:
    texfalse_import()

from Scripts.cluster_scripts import *
import Scripts.extrapolate as extrapolate


# List of DFT functionals used in this study
dft_functionals = {
    "PBE": {"type": "GGA", "dispersion": "None"},
    "PBE-D2": {"type": "GGA", "dispersion": "D2"},
    "PBE-DDSC": {"type": "GGA", "dispersion": "DDSC"},
    "PBE-D3": {"type": "GGA", "dispersion": "D3"},
    "PBE-D3BJ": {"type": "GGA", "dispersion": "D3BJ"},
    "PBE-TS": {"type": "GGA", "dispersion": "TS"},
    "PBE-TSHI": {"type": "GGA", "dispersion": "TSHI"},
    "PBE-D4": {"type": "GGA", "dispersion": "D4"},
    "PBEsol": {"type": "GGA", "dispersion": "None"},
    "PBEsol-D3BJ": {"type": "GGA", "dispersion": "D3BJ"},
    "PBEsol-D4": {"type": "GGA", "dispersion": "D4"},
    "vdW-DF": {"type": "GGA", "dispersion": "vdW"},
    "vdW-DF2": {"type": "GGA", "dispersion": "vdW"},
    "optB86b-vdW": {"type": "GGA", "dispersion": "vdW"},
    "rev-vdW-DF2": {"type": "GGA", "dispersion": "rev-vdW"},
    "TPSS": {"type": "Meta-GGA", "dispersion": "None"},
    "R2SCAN": {"type": "Meta-GGA", "dispersion": "None"},
    "R2SCAN-D3": {"type": "Meta-GGA", "dispersion": "D3"},
    "R2SCAN-D4": {"type": "Meta-GGA", "dispersion": "D4"},
    "SCAN-rVV10": {"type": "Meta-GGA", "dispersion": "rVV10"},
    "PBE0": {"type": "Hybrid", "dispersion": "None"},
    "PBE0-D3": {"type": "Hybrid", "dispersion": "D3"},
    "PBE0-D3BJ": {"type": "Hybrid", "dispersion": "D3BJ"},
    "PBE0-D4": {"type": "Hybrid", "dispersion": "D4"},
    "HSE06": {"type": "Hybrid", "dispersion": "None"},
    "HSE06-D4": {"type": "Hybrid", "dispersion": "D4"},
    "B3LYP": {"type": "Hybrid", "dispersion": "None"},
    "B3LYP-D2": {"type": "Hybrid", "dispersion": "D2"},
    "B3LYP-D3": {"type": "Hybrid", "dispersion": "D3"},
    "B3LYP-D3BJ": {"type": "Hybrid", "dispersion": "D3BJ"},
    "B3LYP-D4": {"type": "Hybrid", "dispersion": "D4"},
    "B2PLYP": {"type": "Double Hybrid", "dispersion": "None"},
    "HF": {"type": "Hartree-Fock", "dispersion": "None"},
    "MP2": {"type": "Post-HF", "dispersion": "None"},
    "CCSD": {"type": "Post-HF", "dispersion": "None"},
    "CCSD(T)": {"type": "Post-HF", "dispersion": "None"}
}


# Some basic conversion factors
kcalmol_to_meV = kcal / mol * 1000
kjmol_to_meV = kJ / mol * 1000
mha_to_meV = Hartree

<a id='tables1'></a>
## Table S1 - List of methods benchmarked in this work

In [5]:
df = pd.DataFrame.from_dict(dft_functionals, orient='index')
df

,type,dispersion
PBE,GGA,None
PBE-D2,GGA,D2
PBE-DDSC,GGA,DDSC
PBE-D3,GGA,D3
PBE-D3BJ,GGA,D3BJ
PBE-TS,GGA,TS
PBE-TSHI,GGA,TSHI
PBE-D4,GGA,D4
PBEsol,GGA,None
PBEsol-D3BJ,GGA,D3BJ


<a id='tables1'></a>
## Benchmarking reliability of CCSD(T)

### Table S2 - Comparison against CCSDT(Q) for gas-phase tetramers

In [6]:
# We compare the relative energies of gas-phase tetramers between CCSD(T) and CCSDT(Q) with the def2-SV(P) basis set
# TODO: Update with def2-SV(P) basis set data rather than LANL2DZ

# Initialize the dictionary for comparing ccsdt and ccsdtq
# ccsdtq_tetramer_ene = {y: {x: {2:0, 3:0, 4:0} for x in ['Au','Ag','Cu']} for y in ['ccsdt','ccsdtq','diff']}
ccsdtq_tetramer_ene = {y: {x: {'ccsdt': 0, 'ccsdtq': 0 , 'diff': 0} for x in ['Au','Ag','Cu']}  for y in [2,3,4]}

# Loop over the 4 Au, Ag, and Cu tetramer geometries, getting energy relative to the first geometry of the appropriate coinage metal
for i in ['Au','Ag','Cu']:
    for j in [1,2,3,4]:
        # Get the relative energy at the CCSDT(Q) level
        ene_ccsdtq = find_energy(f"Data/06-CCSDTQ/CCSDTQ/{i}/{j}/mrcc.out",typ="ccsdtq")
        if j == 1:
            ene0_ccsdtq = ene_ccsdtq
        # Get the relative energy at the CCSD(T) level
        ene_ccsdt = find_energy(f"Data/06-CCSDTQ/CCSDT/{i}/{j}/mrcc.out",typ="ccsdt_tot")
        if j == 1:
            ene0_ccsdt = ene_ccsdt

        if j > 1:
            ccsdtq_tetramer_ene[j][i]['ccsdtq'] = (ene_ccsdtq-ene0_ccsdtq)*Hartree*1000
            ccsdtq_tetramer_ene[j][i]['ccsdt'] = (ene_ccsdt-ene0_ccsdt)*Hartree*1000
            # Get the difference in relative energy between CCSD(T) and CCSDT(Q)
            ccsdtq_tetramer_ene[j][i]['diff'] = ((ene_ccsdtq-ene0_ccsdtq) - (ene_ccsdt-ene0_ccsdt))*Hartree*1000


# Convert the nested dictionary to a MultiIndex DataFrame
df = pd.concat({k: pd.DataFrame(v) for k, v in ccsdtq_tetramer_ene.items()}, axis=0)

# Reset the index to have tetramers (2, 3, 4) as rows
df = df.unstack(level=0)

df = df.round(0).astype(int)

# Adjust column names
df.columns = pd.MultiIndex.from_tuples(df.columns)

df

Au             Ag             Cu                
          2    3    4    2    3    4    2         3     4
ccsdt   211  810  978  253  622  788  328       784  1026
ccsdtq  209  804  961  254  615  779    0 -21242784     0
diff     -2   -7  -18    0   -7   -9 -328 -21243568 -1026

### Table S3 - Using multireference diagnostics to determine accuracy for larger gas-phase clusters

In [17]:
# MR character for Au4, Ag4, Cu4, Au6, Ag6, Cu6, Au8, Cu8, Ag8, Au20
# We use the %Ecorr metric to determine multi-reference character as it is highly robust.

# Initialize the dictionary for MR character
mr_character = {x: {y:0 for y in [1,2,3,4]} for x in ['Au4', 'Ag4', 'Cu4', 'Au6', 'Ag6', 'Cu6', 'Au8', 'Cu8', 'Ag8', 'Au20']}

for i in ['Au4', 'Ag4', 'Cu4', 'Au6', 'Ag6', 'Cu6', 'Au8', 'Cu8', 'Ag8']:
    for j in [1,2,3,4]:

        a1 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/1/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsd')
        a2 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/2/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsd')
        a3 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/3/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsd')
        diff = a1 - a2
        dummy1 = extrapolate.get_cbs(0,a1,0,a2,X=3,Y=4,family='mixcc',output=False)

        a1 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/1/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsdt')
        a2 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/2/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsdt')
        a3 = find_energy('Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{0}/{1}/3/mrcc.out'.format(i,j),code_format='mrcc',typ='ccsdt')
        diff = a1 - a2
        dummy2 = extrapolate.get_cbs(0,a1,0,a2,X=3,Y=4,family='mixcc',output=False)
        mr_character[i][j] = 100*dummy1[-1]/dummy2[-1]

for i in [1,2]:
    a1 = find_energy('Data/04-M20_MgO/CC/{0}/1/mrcc.out'.format(i),code_format='mrcc',typ='ccsd')
    a2 = find_energy('Data/04-M20_MgO/CC/{0}/1/mrcc.out'.format(i),code_format='mrcc',typ='ccsdt')
    mr_character['Au20'][i] = 100* a1/a2

df = df.round(2)


df = pd.DataFrame.from_dict(mr_character)

df

,Au4,Ag4,Cu4,Au6,Ag6,Cu6,Au8,Cu8,Ag8,Au20
1,95.561793,96.009394,95.950028,95.522517,95.939187,95.870435,95.514218,95.852328,95.937314,95.518585
2,95.659767,96.057493,96.030513,95.420158,95.874193,95.802547,95.320740,95.662554,95.788868,95.482674
3,95.660467,96.073886,96.111980,95.404388,95.878280,95.742283,95.408024,95.760279,95.876506,0.000000
4,95.611518,96.034629,95.882717,95.389307,95.893098,95.739554,95.273207,95.626241,95.744767,0.000000


### Table S3 - Benchmarking the chosen LNO settings against canonical CCSD(T)

In [27]:
# We compute the relative energy of the gas-phase tetramers with LNO-CCSD(T) and DF-CCSD(T)
# TODO: Add the leftover DF-CCSD(T) data

# Initialize the dictionary for comparing lno and df
lno_df_tetramer_ene = {y: {x: {'lno': 0, 'df': 0 , 'diff': 0} for x in ['Au','Ag','Cu']}  for y in [2,3,4]}

# Loop over the 4 Au, Ag, and Cu tetramer geometries, getting energy relative to the first geometry of the appropriate coinage metal
for i in ['Au']: #,'Ag','Cu']:
    for j in [2,3,4]:
        # First get the DF-CCSD(T) relative energy
        ene_hf = []
        ene_ccsdt = []
        for k in ['TZ','QZ']:
            ene_hf += [find_energy(f'Data/01-Gas_Cluster/cWFT/DF-CCSDT/{i}4/{j}/{k}/0/mrcc.out',code_format='mrcc',typ='hf') - find_energy(f'Data/01-Gas_Cluster/cWFT/DF-CCSDT/{i}4/1/{k}/0/mrcc.out',code_format='mrcc',typ='hf')]
            ene_ccsdt += [find_energy(f'Data/01-Gas_Cluster/cWFT/DF-CCSDT/{i}4/{j}/{k}/0/mrcc.out',code_format='mrcc',typ='ccsdt') - find_energy(f'Data/01-Gas_Cluster/cWFT/DF-CCSDT/{i}4/1/{k}/0/mrcc.out',code_format='mrcc',typ='ccsdt')]

        extrap_rel_ene = extrapolate.get_cbs(ene_hf[0], ene_ccsdt[0], ene_hf[1], ene_ccsdt[1], X=3, Y=4, family='mixcc',output=False)
        lno_df_tetramer_ene[j][i]['df'] = extrap_rel_ene[-1]*Hartree*1000

        # Now, let's get the energy with LNO-CCSD(T)
        ene_hf = []
        ene_ccsdt = []
        for k in [1,2,3]:
            ene_hf += [find_energy(f'Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{i}4/{j}/{k}/mrcc.out',code_format='mrcc',typ='hf') - find_energy(f'Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{i}4/1/{k}/mrcc.out',code_format='mrcc',typ='hf')]
            ene_ccsdt += [find_energy(f'Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{i}4/{j}/{k}/mrcc.out',code_format='mrcc',typ='ccsdt') - find_energy(f'Data/01-Gas_Cluster/cWFT/LNO-CCSDT/{i}4/1/{k}/mrcc.out',code_format='mrcc',typ='ccsdt')]

        extrap_rel_ene = extrapolate.get_cbs(ene_hf[0], ene_ccsdt[0], ene_hf[2], ene_ccsdt[2], X=3, Y=4, family='mixcc',output=False)
        lno_correction = ene_hf[1] + ene_ccsdt[1] - (ene_hf[0] + ene_ccsdt[0])
        lno_df_tetramer_ene[j][i]['lno'] = (extrap_rel_ene[-1] + lno_correction)*Hartree*1000
        lno_df_tetramer_ene[j][i]['diff'] = lno_df_tetramer_ene[j][i]['lno'] - lno_df_tetramer_ene[j][i]['df']

# Convert the nested dictionary to a MultiIndex DataFrame
df = pd.concat({k: pd.DataFrame(v) for k, v in lno_df_tetramer_ene.items()}, axis=0)

# Reset the index to have tetramers (2, 3, 4) as rows
df = df.unstack(level=0)

df = df.round(0).astype(int)

# Adjust column names
df.columns = pd.MultiIndex.from_tuples(df.columns)

df
    

Au            Ag       Cu      
        2    3     4  2  3  4  2  3  4
lno   143  687  1183  0  0  0  0  0  0
df    148  769  1229  0  0  0  0  0  0
diff   -5  -81   -46  0  0  0  0  0  0

# Benchmarking the SKZCAM protocol

### Figure S1 - Convergence of the dimer on MgO adsorption energy as a function of cluster size

In [42]:
# Dimers

# Initialize the dictionary for the convergence of adsorption energy with size of the cluster
dimer_size_ene_list = {x: [] for x in range(1,9)}

# This is the dictionary that connects the cluster index with the corresponding orientation/positions (e.g., OV, MV, etc.)
# We give 

dimer_dict = {
    1: ['OV', 5],
    2: ['MV', 6],
    3: ['HV', 3],
    4: ['HD', 6],
    5: ['OH', 4],
    6: ['MH', 5],
    7: ['OD', 4],
    8: ['MD', 5]
}

# Loop over each dimer structure
for i in range(1,9):
    for j in range(1,10):
        a = find_energy('Data/02-M2_MgO/cWFT/CC/Size_Convergence/{0}/{1}/AD_SLAB/mrcc.out'.format(i,j),typ='lccsdt_tot')
        b = find_energy('Data/02-M2_MgO/cWFT/CC/Size_Convergence/{0}/{1}/AD_CP/mrcc.out'.format(i,j),typ='lccsdt_tot')
        c = find_energy('Data/02-M2_MgO/cWFT/CC/Size_Convergence/{0}/{1}/SLAB_CP/mrcc.out'.format(i,j),typ='lccsdt_tot')
        dimer_size_ene_list[i] += [(a-b-c)*Hartree*1000]

ene_list_errors = {x: [] for x in range(1,9)}
dimer_size_mad_error = {dimer_dict[x][0]: 0 for x in range(1,9)}

# Calculate the error of the chosen cluster w.r.t. larger cluster sizes
for i in range(1,9):
    for j in range(1,10):
        if j > dimer_dict[i][1]:
            ene_list_errors[i] += [dimer_size_ene_list[i][j-1] - dimer_size_ene_list[i][dimer_dict[i][1]-1]]

    dimer_size_mad_error[dimer_dict[i][0]] = np.mean(np.abs(ene_list_errors[i]))




In [65]:
fig, axs = plt.subplots(4,2,figsize=(6.69,9),dpi=450,sharex=True,constrained_layout=True)

for i in range(1,9):
    b = divmod(i-1,4)[0]
    a = divmod(i-1,4)[1]
    axs[a,b].scatter(list(range(1,10)),dimer_size_ene_list[i], marker = 'x',color=color_dict['blue'])
    axs[a,b].scatter(dimer_dict[i][1],dimer_size_ene_list[i][dimer_dict[i][1]-1], s=150,marker = 'o', color = color_dict['red'], alpha = 0.5,edgecolor=None)
    axs[a,b].set_ylim([dimer_size_ene_list[i][-1] -100, dimer_size_ene_list[i][-1] + 150])
    axs[a,b].set_title('Dimer {0}'.format(dimer_dict[i][0]))
    axs[a,b].set_xlim([0,10])
    axs[a,b].set_xticks(list(range(1,10)))
    if b ==0:
        axs[a,b].set_ylabel(r'$E_\textrm{bind}$ (meV)')
    if a == 3:
        axs[a,b].set_xlabel('Cluster from SKZCAM protocol')

plt.savefig('Figures_Final/Fig_S1.png')

### Figure S2 - Convergence of the relative energy of tetramers on MgO as a function of cluster size

In [ ]:
# Tetramers

tetramer_size_ene_list = {x: {y: {z: [] for z in ['DFMP2','LMP2_normal','LMP2_tight']} for y in [1,2,3,4]} for x in ['Au','Ag','Cu']}

for i in ['Au','Ag','Cu']:
    for j in [1,2,3,4]:
        for k in ['DFMP2','LMP2_normal','LMP2_tight']:
            for l in range(1,9):
                a0 = find_energy('Data/03-M4_MgO/cWFT/CCSDT/Size_Convergence/{0}/{1}/{2}/{3}/AD_SLAB/mrcc.out'.format(i,1,l,k),code_format='mrcc',typ='lmp2_tot')
                a = find_energy('Data/03-M4_MgO/cWFT/CCSDT/Size_Convergence/{0}/{1}/{2}/{3}/AD_SLAB/mrcc.out'.format(i,j,l,k),code_format='mrcc',typ='lmp2_tot')
                if np.abs(a-a0) < 0.0001:
                    tetramer_size_ene_list[i][j][k] += [10000]
                else:
                    tetramer_size_ene_list[i][j][k] += [(a-a0)*Hartree*1000]

fig, axs = plt.subplots(3,3,figsize=(6.69,7),dpi=450,sharex=True,constrained_layout=True)


for index,i in enumerate(['Au','Ag','Cu']):
    for j in [2,3,4]:
        axs[j-2,index].scatter(list(range(1,9)),tetramer_size_ene_list[i][j]['DFMP2'], marker = 'x',color=color_dict['blue'],alpha=0.7,label='MP2')
        axs[j-2,index].scatter(list(range(1,9)),tetramer_size_ene_list[i][j]['LMP2_normal'], marker = 'o',color=color_dict['red'],alpha=0.7,facecolor = 'none',label = 'LMP2 (normal)')
        axs[j-2,index].scatter(list(range(1,9)),tetramer_size_ene_list[i][j]['LMP2_tight'], marker = 's',color=color_dict['green'],alpha=0.7, facecolor='none', label = 'LMP2 (tight)')


        # axs[j,index].scatter(dimer_dict[i][1],dimer_size_ene_list[i][dimer_dict[i][1]-1], s=150,marker = 'o', color = color_dict['red'], alpha = 0.5,edgecolor=None)
        axs[j-2,index].set_ylim([tetramer_size_ene_list[i][j]['LMP2_normal'][-1] -200, tetramer_size_ene_list[i][j]['LMP2_normal'][-1] + 250])
        axs[j-2,index].set_title(f'{i} Tetramer Structure {j}')
        axs[j-2,index].set_xlim([0,10])
        axs[j-2,index].set_xticks(list(range(1,10)))
        if index ==0:
            axs[j-2,index].set_ylabel('Rel. Eneregy to Structure 1 (meV)') #([0,10])

            # axs[a,b].set_ylabel(r'$E_\textrm{bind}$ (meV)')
        if j == 4:
            axs[j-2,index].set_xlabel('Cluster from SKZCAM protocol')

axs[0,0].legend()

plt.savefig('Figures_Final/Fig_S2.png')


In [68]:
tetramer_size_ene_list[i][j]

{'DFMP2': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'LMP2_normal': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'LMP2_tight': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}